In [2]:
# Import modules
from selenium import webdriver
from selenium.webdriver.common.by import By
import smtplib
import time
import configparser
import pandas as pd
import uuid
from datetime import datetime
import json
import os

# Set maximum number of rows and columns to display
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [16]:
with open('/mnt/c/Users/mahei/OneDrive/Documentos/Python/monitor-aliexpress-product-price/products.json', 'r') as f:
    products = json.load(f)

In [40]:
# Load the parquet file
file_path = './prices.parquet'
previous_prices = pd.read_parquet(file_path, engine='fastparquet')

# Extract year, month, and day as new columns
previous_prices['year'] = previous_prices['datetime'].dt.year
previous_prices['month'] = previous_prices['datetime'].dt.month
previous_prices['day'] = previous_prices['datetime'].dt.day

# Save the data with year, month, and day partitions
previous_prices.to_parquet('./data/prices.parquet', engine='fastparquet', compression='gzip', partition_cols=['product', 'year', 'month', 'day'])

In [3]:
df = pd.read_parquet('./data/prices.parquet', filters=[('product', '==', 'Ryzen 7 5800X3D')])


In [5]:
df.sort_values(by=['datetime'], ascending=False)

,id,loop_id,url,price,datetime,product,year,month,day
1001,61d4df74-36e5-48cd-83e6-ac0aa682f531,0833fe91-5e4d-4109-bd04-adb260752c1b,https://pt.aliexpress.com/item/100500524029125...,2690.15,2023-03-06 15:53:21.460396,Ryzen 7 5800X3D,2023,3,6
1000,e4a456d7-620e-4138-9199-0018491526fb,0833fe91-5e4d-4109-bd04-adb260752c1b,https://pt.aliexpress.com/item/100500524021876...,2690.15,2023-03-06 15:53:14.709996,Ryzen 7 5800X3D,2023,3,6
999,b4d92b24-9abd-4da4-b88a-5291ebce72d5,0833fe91-5e4d-4109-bd04-adb260752c1b,https://pt.aliexpress.com/item/100500490878567...,2404.61,2023-03-06 15:53:13.310486,Ryzen 7 5800X3D,2023,3,6
998,b9437f9c-1f28-41d3-9220-894bd8b1a14b,0833fe91-5e4d-4109-bd04-adb260752c1b,https://pt.aliexpress.com/item/100500497046519...,1985.84,2023-03-06 15:53:11.878769,Ryzen 7 5800X3D,2023,3,6
997,ccddf8fd-4b1f-4872-86a3-6051e47b86f4,0833fe91-5e4d-4109-bd04-adb260752c1b,https://pt.aliexpress.com/item/100500497226382...,2096.74,2023-03-06 15:53:10.627223,Ryzen 7 5800X3D,2023,3,6
...,...,...,...,...,...,...,...,...,...
4,a9bbf655-e7da-48b7-9f5d-dab6197b3abe,96f22f75-fe8e-45f8-988a-4beca2efc762,https://pt.aliexpress.com/item/100500524021876...,2687.76,2023-03-02 12:42:16.234405,Ryzen 7 5800X3D,2023,3,2
3,966f416a-e420-416d-8e6b-198db6dea9cc,96f22f75-fe8e-45f8-988a-4beca2efc762,https://pt.aliexpress.com/item/100500490878567...,2396.81,2023-03-02 12:42:13.865165,Ryzen 7 5800X3D,2023,3,2
2,bdd6df65-f4c9-4d3b-b097-ec9c3c613282,96f22f75-fe8e-45f8-988a-4beca2efc762,https://pt.aliexpress.com/item/100500497046519...,2001.64,2023-03-02 12:42:11.616478,Ryzen 7 5800X3D,2023,3,2
1,a48961b1-9ec2-4448-83ae-47690c98ef4f,96f22f75-fe8e-45f8-988a-4beca2efc762,https://pt.aliexpress.com/item/100500497226382...,1946.96,2023-03-02 12:42:09.655925,Ryzen 7 5800X3D,2023,3,2


In [8]:
# Sort the dataframe by 'datetime' column in ascending order
df = df.sort_values('datetime')

In [13]:
df['prev_price'] = df.groupby(['product', 'url'])['price'].shift(1)
df['price_diff'] = df['price'] - df['prev_price']
df = df[df["price_diff"] != 0]

In [ ]:
df

In [34]:
df.groupby(['product', df['datetime'].dt.date]).agg({'price':'min'})

price
product         datetime           
Ryzen 7 5800X3D 2023-03-02  1790.77
                2023-03-04  1927.42
                2023-03-06  1921.76
Ryzen 9 5950X   2023-03-02  2640.06
                2023-03-04  2650.11
                2023-03-06  2642.40